In [2]:
# train.py (first few lines)
import os, shutil

SRC = 'train_zip/train'
IMG_DIR = 'data/images'
ANN_DIR = 'data/annotations'
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(ANN_DIR, exist_ok=True)

for fname in os.listdir(SRC):
    src_path = os.path.join(SRC, fname)
    if fname.lower().endswith(('.jpg','jpeg','png')):
        shutil.copy(src_path, IMG_DIR)
    elif fname.lower().endswith('.xml'):
        shutil.copy(src_path, ANN_DIR)


# import lib

In [ ]:
import os, xml.etree.ElementTree as ET, yaml, cv2
from ultralytics import YOLO
from pathlib import Path
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

In [ ]:
SRC_IMG = 'data/images'
SRC_XML = 'data/annotations'
YOLO_IMG = 'yolo_data/images'
YOLO_XML = 'yolo_data/annotations'

In [ ]:
os.makedirs(YOLO_IMG, exist_ok=True)
os.makedirs(YOLO_XML, exist_ok=True)

In [ ]:
for xml_file in os.listdir(SRC_XML):
    """    
    convert Pascal VOC XML to YOLO format
    """
    if not xml_file.endswith('.xml'): continue
    tree = ET.parse(os.path.join(SRC_XML, xml_file))
    root = tree.getroot()
    img_name = xml_file.replace('.xml', '.jpg')
    img_path = os.path.join(SRC_IMG, img_name)
    img = cv2.imread(img_path)
    h, w = img.shape[:2]
    
    cv2.imwrite(os.path.join(YOLO_IMG, img_name), img)
  
    lines = []
    for obj in root.findall('object'):
        if obj.find('name').text.lower() != 'apple': continue
        b = obj.find('bndbox')
        x1, y1, x2, y2 = [int(b.find(k).text) for k in ('xmin','ymin','xmax','ymax')]
        cx, cy = ((x1+x2)/2)/w, ((y1+y2)/2)/h
        bw, bh = (x2-x1)/w, (y2-y1)/h
        lines.append(f"0 {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")
    
    with open(os.path.join(YOLO_LBL, img_name.replace('.jpg','.txt')), 'w') as f:
        f.write("\n".join(lines))


In [ ]:
# 5) write data.yaml with absolute paths so YOLOv8 can find your local folders
train_path = Path(YOLO_IMG).resolve().as_posix()
val_path   = train_path  # or split out a separate val folder

In [ ]:
data_cfg = {
    'train': train_path,
    'val':   val_path,
    'nc':    1,
    'names': ['apple']
}
with open('data.yaml','w') as f:
    yaml.dump(data_cfg, f)

In [ ]:
# 6) train YOLOv8n for 20 epochs
model = YOLO('yolov8n.pt')
model.train(data='data.yaml', epochs=20, imgsz=640)


In [ ]:
# 7) run inference on the first image & plot results inline
first_img = next(Path(YOLO_IMG).glob('*.jpg'))
results   = model.predict(source=str(first_img), conf=0.5)[0]
img_pil   = Image.open(first_img).convert('RGB')
draw      = ImageDraw.Draw(img_pil)
for box, cls, conf in zip(results.boxes.xyxy, results.boxes.cls, results.boxes.conf):
    x1, y1, x2, y2 = box
    draw.rectangle([x1, y1, x2, y2], outline='red', width=2)
    draw.text((x1, y1-10), f"Apple {conf:.2f}", fill='red')

plt.figure(figsize=(6,6))
plt.imshow(img_pil)
plt.axis('off')

In [ ]:







SRC_IMG, SRC_XML = 'data/images', 'data/annotations'
YOLO_IMG, YOLO_LBL = 'yolo_dataset/images', 'yolo_dataset/labels'
os.makedirs(YOLO_IMG, exist_ok=True)
os.makedirs(YOLO_LBL, exist_ok=True)

# 4) Pascal VOC (.xml) → YOLO (.txt) conversion
for xml_file in os.listdir(SRC_XML):
    if not xml_file.endswith('.xml'): continue
    tree = ET.parse(os.path.join(SRC_XML, xml_file))
    root = tree.getroot()
    img_name = xml_file.replace('.xml', '.jpg')
    img_path = os.path.join(SRC_IMG, img_name)
    img = cv2.imread(img_path)
    h, w = img.shape[:2]
    # copy image
    cv2.imwrite(os.path.join(YOLO_IMG, img_name), img)
    # build YOLO label lines
    lines = []
    for obj in root.findall('object'):
        if obj.find('name').text.lower() != 'apple': continue
        b = obj.find('bndbox')
        x1, y1, x2, y2 = [int(b.find(k).text) for k in ('xmin','ymin','xmax','ymax')]
        cx, cy = ((x1+x2)/2)/w, ((y1+y2)/2)/h
        bw, bh = (x2-x1)/w, (y2-y1)/h
        lines.append(f"0 {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")
    # save label
    with open(os.path.join(YOLO_LBL, img_name.replace('.jpg','.txt')), 'w') as f:
        f.write("\n".join(lines))

# 5) write data.yaml with absolute paths so YOLOv8 can find your local folders
train_path = Path(YOLO_IMG).resolve().as_posix()
val_path   = train_path  # or split out a separate val folder
data_cfg = {
    'train': train_path,
    'val':   val_path,
    'nc':    1,
    'names': ['apple']
}
with open('data.yaml','w') as f:
    yaml.dump(data_cfg, f)



# 6) train YOLOv8n for 20 epochs
model = YOLO('yolov8n.pt')
model.train(data='data.yaml', epochs=20, imgsz=640)

# 7) run inference on the first image & plot results inline
first_img = next(Path(YOLO_IMG).glob('*.jpg'))
results   = model.predict(source=str(first_img), conf=0.5)[0]
img_pil   = Image.open(first_img).convert('RGB')
draw      = ImageDraw.Draw(img_pil)
for box, cls, conf in zip(results.boxes.xyxy, results.boxes.cls, results.boxes.conf):
    x1, y1, x2, y2 = box
    draw.rectangle([x1, y1, x2, y2], outline='red', width=2)
    draw.text((x1, y1-10), f"Apple {conf:.2f}", fill='red')

plt.figure(figsize=(6,6))
plt.imshow(img_pil)
plt.axis('off')


===== data.yaml =====
names:
- apple
nc: 1
train: C:/Users/Admin/Internship/projects/Apple Detection/yolo_dataset/images
val: C:/Users/Admin/Internship/projects/Apple Detection/yolo_dataset/images

Ultralytics 8.3.102  Python-3.9.0 torch-2.6.0+cpu CPU (AMD Ryzen 7 7730U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=Fal

100%|██████████| 755k/755k [00:00<00:00, 1.18MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 21                  -1  1    493056  ultralytics.nn.modules.block.C2f             [384, 256, 1]                 
 22        [15, 18, 21]  1    751507  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'


train: Scanning C:\Users\Admin\Internship\projects\Apple Detection\yolo_dataset\labels... 240 images, 150 backgrounds, 0 corrupt: 100%|██████████| 240/240 [00:00<00:00, 473.59it/s]

...

                   all        240        156      0.956      0.968       0.99      0.826



20 epochs completed in 0.530 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 6.2MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.102  Python-3.9.0 torch-2.6.0+cpu CPU (AMD Ryzen 7 7730U with Radeon Graphics)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:17<00:00,  2.17s/it]


                   all        240        156      0.956      0.964       0.99      0.827
Speed: 1.8ms preprocess, 63.0ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\train2

image 1/1 c:\Users\Admin\Internship\projects\Apple Detection\yolo_dataset\images\apple_1.jpg: 640x640 1 apple, 76.7ms
Speed: 3.2ms preprocess, 76.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


(np.float64(-0.5), np.float64(348.5), np.float64(348.5), np.float64(-0.5))

ModuleNotFoundError: No module named 'tflite_model_maker'